# Bronze Layer
medallion_staging --> Bronze Table


##Loading Cutsomers Data

In [0]:
spark.read.csv(
    path="/Volumes/dev_cagtalog/default/retail_staging/customers/customers.csv",
    header=True,
    inferSchema=True,
).write.format("delta").mode("overwrite").saveAsTable("dev_cagtalog.bronze.customers")

In [0]:
spark.read.table("dev_cagtalog.bronze.customers").display()

##Load Order Data

In [0]:
spark.read.csv(
    path="/Volumes/dev_cagtalog/default/retail_staging/orders/orders_001.csv",
    header=True,
    inferSchema=True,
).write.format("delta").mode("overwrite").saveAsTable("dev_cagtalog.bronze.orders_001")

In [0]:
spark.read.table("dev_cagtalog.bronze.orders_001").display()

#Silver

In [0]:
from pyspark.sql.functions import col
 
spark.read.table("dev_cagtalog.bronze.customers").filter(
    col("CustomerName").isNotNull()
).write.mode("append").saveAsTable("dev_cagtalog.silver.customers")
spark.read.table("dev_cagtalog.silver.customers").display()

In [0]:
from pyspark.sql.functions import col
 
spark.read.table("dev_cagtalog.bronze.orders_001").filter(
    col("CustomerID").isNotNull()
).write.mode("append").saveAsTable("dev_cagtalog.silver.orders_001")
spark.read.table("dev_cagtalog.silver.orders_001").display()

#Gold


In [0]:
customers_df = spark.read.table("dev_cagtalog.silver.customers")
orders_df = spark.read.table("dev_cagtalog.silver.orders_001")
joined_df = customers_df.join(
    orders_df, customers_df.CustomerID == orders_df.CustomerID, how = "inner"
)
joined_df.select(customers_df.CustomerID,"OrderID").groupBy("CustomerID").count().write.mode("append").saveAsTable("dev_cagtalog.gold.sales_agg")
spark.read.table("dev_cagtalog.gold.sales_agg").display()